In [1]:
import torch

# Create a tensor
x = torch.tensor([[1, 2],
                  [3, 4]])

print(x)
print(x.shape)


tensor([[1, 2],
        [3, 4]])
torch.Size([2, 2])


In [2]:

import shutil, os

os.makedirs("/content/data/flower", exist_ok=True)

shutil.copy(
    "/content/flower1.jpg",
    "/content/data/flower/flower1.jpg"
)



'/content/data/flower/flower1.jpg'

In [3]:
from torch.utils.data import Dataset
from PIL import Image
import os

class SimpleImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.images = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        image = Image.open(img_path).convert("RGB")
        label = 0  # dummy label for now

        if self.transform:
            image = self.transform(image)

        return image, label


In [4]:
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

dataset = SimpleImageDataset("/content/data/flower", transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)


In [5]:
import torch
import torchvision.models as models
from torchvision.models import ResNet18_Weights

model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = torch.nn.Identity()
model.eval()

images, labels = next(iter(dataloader))

with torch.no_grad():
    features = model(images)

print("Image batch:", images.shape)
print("Extracted features:", features.shape)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 118MB/s]


Image batch: torch.Size([1, 3, 224, 224])
Extracted features: torch.Size([1, 512])


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 56 * 56, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [7]:
model = SimpleCNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()

for epoch in range(1):  # 1 epoch for understanding
    for images, labels in dataloader:

        optimizer.zero_grad()        # reset gradients
        outputs = model(images)      # forward pass
        loss = criterion(outputs, labels)  # compute loss
        loss.backward()              # backpropagation
        optimizer.step()             # update weights

        print("Loss:", loss.item())

model.eval()

with torch.no_grad():
    for images, labels in dataloader:
        outputs = model(images)

Loss: 0.626800000667572
